# Preparing Data

In [2]:
import pandas as pd

plays_path = '/Users/davidstange/Development/github/davestange/data/big_data_bowl_2025/plays.csv'

In [58]:
plays = pd.read_csv(plays_path)

plays['offenseFormation'].unique() # 'EMPTY', 'SHOTGUN', 'PISTOL', 'SINGLEBACK', nan, 'JUMBO', 'I_FORM', 'WILDCAT'
plays['receiverAlignment'].unique() # '3x2', '2x2', '3x1', nan, '4x1', '2x1', '1x1', '2x0', '3x0', '1x0', '4x2', '3x3'
plays['passResult'].unique() # 

tv_watching = ['gameId', 'playId', 'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'preSnapHomeScore', 'preSnapVisitorScore']

steelers_bengals = plays[plays.gameId == 2022091103].sort_values(by='playId')[tv_watching]
pre_snap = plays[['gameId', 'playId', 'quarter', 'down', 'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore', 'absoluteYardlineNumber', 'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability', 'offenseFormation', 'receiverAlignment', 'playClockAtSnap']]

# -------- IDENTIFIERS
# plays['gameId'].unique() # gameId
# plays['playId'].unique() # playId


# -------- PRE-SNAP DETAILS
# plays['quarter'].unique() # quarter
# plays['down'].unique() # down
# plays['yardsToGo'].unique() # yardsToGo
# plays['possessionTeam'].unique() # possessionTeam
# plays['defensiveTeam'].unique() # defensiveTeam
# plays['yardlineSide'].unique()            # what does this mean? which side of the field they are on?
# plays['yardlineNumber'].unique()          # 1-50
# plays['gameClock'].unique() # gameClock
# plays['preSnapHomeScore'].unique() # preSnapHomeScore
# plays['preSnapVisitorScore'].unique() # preSnapVisitorScore
# plays['absoluteYardlineNumber'].unique()  # 1-99
# plays['preSnapHomeTeamWinProbability'].unique() # preSnapHomeTeamWinProbability
# plays['preSnapVisitorTeamWinProbability'].unique() # preSnapVisitorTeamWinProbability
# plays['offenseFormation'].unique()        # 'EMPTY', 'SHOTGUN', 'PISTOL', 'SINGLEBACK', nan, 'JUMBO', 'I_FORM', 'WILDCAT'
# plays['receiverAlignment'].unique()       # '3x2', '2x2', '3x1', nan, '4x1', '2x1', '1x1', '2x0', '3x0', '1x0', '4x2', '3x3'
# plays['playClockAtSnap'].unique()         # playClockAtSnap


# -------- PLAY TYPE
# plays['qbSpike'].unique() # qbSpike
# plays['qbKneel'].unique() # qbKneel
# plays['qbSneak'].unique() # qbSneak
# plays['isDropback'].unique() # isDropback


# -------- POST-SNAP RESULT
# plays['playDescription'].unique() # playDescription



# plays['playNullifiedByPenalty'].unique() # playNullifiedByPenalty
# plays['expectedPoints'].unique() # expectedPoints
# plays['passResult'].unique() # passResult
# plays['passLength'].unique() # passLength
# plays['targetX'].unique() # targetX
# plays['targetY'].unique() # targetY
# plays['playAction'].unique() # playAction
# plays['dropbackType'].unique() # dropbackType
# plays['dropbackDistance'].unique() # dropbackDistance
# plays['passLocationType'].unique() # passLocationType
# plays['timeToThrow'].unique() # timeToThrow
# plays['timeInTackleBox'].unique() # timeInTackleBox
# plays['timeToSack'].unique() # timeToSack
# plays['passTippedAtLine'].unique() # passTippedAtLine
# plays['unblockedPressure'].unique() # unblockedPressure
# plays['rushLocationType'].unique()        # nan, 'INSIDE_LEFT', 'INSIDE_RIGHT', 'OUTSIDE_LEFT', 'UNKNOWN', 'OUTSIDE_RIGHT'
# plays['penaltyYards'].unique() # penaltyYards
# plays['prePenaltyYardsGained'].unique() # prePenaltyYardsGained
# plays['yardsGained'].unique() # yardsGained
# plays['homeTeamWinProbabilityAdded'].unique() # homeTeamWinProbabilityAdded
# plays['visitorTeamWinProbilityAdded'].unique() # visitorTeamWinProbilityAdded
# plays['expectedPointsAdded'].unique() # expectedPointsAdded
# plays['pff_runConceptPrimary'].unique()   # nan, 'MAN', 'INSIDE ZONE', 'UNDEFINED', 'PULL LEAD', 'OUTSIDE ZONE', 'TRAP', 'SNEAK', 'POWER', 'COUNTER', 'DRAW', 'TRICK', 'FB RUN'
# plays['pff_runConceptSecondary'].unique() # [nan, 'READ OPTION', 'LEAD', 'PITCH', 'INVERTED;READ OPTION', 'SPLIT', 'MISDIRECTION', 'READ OPTION;SPLIT', 'LEAD;PITCH', 'CROSS LEAD;LEAD', 'CROSS LEAD', 'LEAD;SPLIT', 'PITCH;SPLIT',
#                                           # 'READ OPTION;SPEED OPTION', 'LEAD;READ OPTION;SPLIT', 'LEAD;QB RUNS', 'QB RUNS', 'LEAD;READ OPTION', 'CROSS LEAD;READ OPTION', 'CROSS LEAD;QB RUNS', 'MISDIRECTION;PITCH', 'LEAD;SPEED OPTION',
#                                           # 'READ OPTION;SPEED OPTION;SPLIT', 'INVERTED;READ OPTION;SPLIT','SPEED OPTION', 'BACKSIDE FOLD','INVERTED;READ OPTION;SPEED OPTION',
#                                           # 'CROSS LEAD;READ OPTION;SPLIT', 'CROSS LEAD;LEAD;READ OPTION','CROSS LEAD;INVERTED;READ OPTION', 'CROSS LEAD;SPLIT','CROSS LEAD;PITCH', 'CROSS LEAD;MISDIRECTION',
#                                           # 'MISDIRECTION;SPLIT', 'CROSS LEAD;READ OPTION;SPEED OPTION','LEAD;MISDIRECTION', 'QB RUNS;SPLIT', 'MISDIRECTION;QB RUNS','LEAD;READ OPTION;SPEED OPTION', 'LEAD;PITCH;SPLIT',
#                                           # 'CROSS LEAD;LEAD;QB RUNS', 'BACKSIDE FOLD;QB RUNS','BACKSIDE FOLD;PITCH', 'LEAD;LEAD']
# plays['pff_runPassOption'].unique()       # 0, 1
# plays['pff_passCoverage'].unique()        # 'Cover-3', 'Quarters', 'Cover-1', 'Cover 6-Left', 'Cover-2', nan, 'Cover-0', 'Cover-3 Seam', 'Cover-6 Right', 'Goal Line', 'Red Zone', '2-Man', 'Cover-3 Double Cloud', 'Bracket',
#                                           # 'Miscellaneous', 'Cover-3 Cloud Left', 'Prevent', 'Cover-1 Double', 'Cover-3 Cloud Right'
# plays['pff_manZone'].unique()             # 'Zone', 'Man', nan, 'Other'

steelers_bengals

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,preSnapHomeScore,preSnapVisitorScore
10263,2022091103,58,(15:00) (Shotgun) M.Trubisky pass short middle...,1,1,10,PIT,CIN,PIT,25,0,0
1278,2022091103,82,"(14:39) (No Huddle, Shotgun) M.Trubisky pass s...",1,2,7,PIT,CIN,PIT,28,0,0
2595,2022091103,111,(14:21) (Shotgun) M.Trubisky pass short left t...,1,3,2,PIT,CIN,PIT,33,0,0
6254,2022091103,176,(13:31) (Shotgun) J.Burrow sacked at CIN 17 fo...,1,1,10,CIN,PIT,CIN,22,0,0
9518,2022091103,195,(12:52) (Shotgun) J.Burrow pass short right in...,1,2,15,CIN,PIT,CIN,17,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6883,2022091103,5015,(:51) (Shotgun) M.Trubisky pass short left to ...,5,2,10,PIT,CIN,PIT,20,20,20
11577,2022091103,5039,"(:34) (No Huddle, Shotgun) M.Trubisky pass dee...",5,3,1,PIT,CIN,PIT,29,20,20
15571,2022091103,5074,(:25) (Shotgun) M.Trubisky pass incomplete sho...,5,1,10,PIT,CIN,CIN,45,20,20
14229,2022091103,5096,(:21) (Shotgun) M.Trubisky pass short middle t...,5,2,10,PIT,CIN,CIN,45,20,20


In [46]:
plays[plays.pff_manZone != 'Zone'][plays.pff_manZone != 'Man'][plays.pff_manZone != 'Other'][['pff_manZone', 'playDescription']]

/var/folders/h4/wyntgh0105x1kw4qjs8wrl840000gn/T/ipykernel_86234/2844629184.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  plays[plays.pff_manZone != 'Zone'][plays.pff_manZone != 'Man'][plays.pff_manZone != 'Other'][['pff_manZone', 'playDescription']]
/var/folders/h4/wyntgh0105x1kw4qjs8wrl840000gn/T/ipykernel_86234/2844629184.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  plays[plays.pff_manZone != 'Zone'][plays.pff_manZone != 'Man'][plays.pff_manZone != 'Other'][['pff_manZone', 'playDescription']]


,pff_manZone,playDescription
7,NaN,(:35) K.Murray kneels to CAR 29 for -1 yards.
25,NaN,(:10) G.Smith kneels to SEA 24 for -1 yards.
53,NaN,(1:16) J.Brissett kneels to CIN 10 for no gain.
173,NaN,(1:31) C.Rush kneels to DAL 38 for -1 yards.
180,NaN,(:05) R.Wilson kneels to DEN 24 for -1 yards.
...,...,...
13671,NaN,(:11) J.Herbert kneels to LAC 34 for -1 yards.
13727,NaN,(:06) A.Rodgers kneels to GB 15 for -1 yards.
13990,NaN,(:18) T.Tagovailoa kneels to MIA 19 for -1 yards.
14556,NaN,(:39) C.Rush kneels to NYG 31 for -1 yards.
